In [ ]:
# Author: Maxime Leurquin
# Date: Nov-2023
# Description: A collection of classes to help creating svd files.

In [ ]:
import os
from win32com.client import Dispatch
import numpy as np
import matplotlib.pyplot as plt

from PolytecController.polyfile import Svd
from PolytecController.enumerations import *

# Offsetting scanpoints of an existing svd file

In [ ]:
def offset_scanpoints(source_filepath,savepath,world_offset):
    """
    Offset must be given in video coordinates (meters)
    """
    #open source .svd file and save it as a .set file
    source_file = Dispatch('PolyFile.PolyFile')
    source_file.Open(source_filepath)
    settings_filepath="my_temp_settings.set"
    source_file.SaveAsEx(PTCSaveAs["ptcSaveAsTypeAllNoOriginal"], settings_filepath)
    source_file.Close()
    
    #Open .set file
    settings_file = Dispatch('PolyFile.PolyFile')
    settings_file.Open(settings_filepath)
    settings_file.ReadOnly = False
    #modify coordinates of settings file.
    video_offset=settings_file.Infos.VideoMappingInfo.CoordXYFromVideo(world_offset[0],world_offset[1])
    print(f"video_offset {video_offset}")
    for measPoint in settings_file.Infos.MeasPoints:
        videoX, videoY = measPoint.VideoXY()
        measPoint.SetVideoXY(videoX +video_offset[0], videoY + video_offset[1])
    settings_file.Save()
    settings_file.SaveAsEx(PTCSaveAs["ptcSaveAsTypeAllNoOriginal"], savepath)
    settings_file.Close()
    os.remove(settings_filepath)
    return

In [ ]:
source_filepath=os.path.join("random_msa_600_settings","my_svd_file.svd")
savepath=os.path.join("random_msa_600_settings","my_settings_modded.set")

offset=np.array([80,0]) #in world coordinates whatever it means
offset_scanpoints(source_filepath,savepath,offset)

fig,ax=plt.subplots(figsize=(10,10))
original_svd=Svd(source_filepath)
original_svd.get_image(show_scanpoints=True,ax=ax)
original_svd.close_file()

modded_svd=Svd(savepath)
modded_scanpoints=modded_svd.get_scanpoints(category="all",ref_frame="pixel")
modded_svd.close_file()

ax.scatter(modded_scanpoints[:,0],modded_scanpoints[:,1],label="modded",s=5,color="orange")
ax.legend()
ax.grid()
fig.show()

# Polytec problem demonstration
Adding measurement points is not possible. Saving the file deletes its contents.

In [ ]:
import os,shutil
from win32com.client import Dispatch

source_filepath=os.path.join("random_msa_600_settings","dv_settings_line.set")
copy_filepath=os.path.join("random_msa_600_settings","dv_settings_line_copy.set")
saved_filepath=os.path.join("random_msa_600_settings","dv_settings_line_edited.set")
shutil.copyfile(source_filepath, copy_filepath)

#open copied file
file = Dispatch('PolyFile.PolyFile')
file.open(copy_filepath)
file.Readonly=False
file.Save()
print(f"Original file: HasMaspoints: {file.Infos.HasMeasPoints}, count: {file.Infos.MeasPoints.Count}")

#add new measpoints:
list_of_points=[(10,10),(9,9),(8,8)]
for point in list_of_points:
    #add points to the file:
    new_measpoint = file.Infos.MeasPoints.Add()
    new_measpoint.SetVideoXY(point[0], point[1])
file.Save()
print(f"After adding new points: HasMaspoints: {file.Infos.HasMeasPoints}, count: {file.Infos.MeasPoints.Count}")

#save the file and close it.
file.SaveAsEx(PTCSaveAs=508,bstrFileNameDest=saved_filepath) #ptcSaveAsTypeAll=508
file.Close()

#open the file we just saved and check contents:
file = Dispatch('PolyFile.PolyFile')
file.open(saved_filepath)
print(f"edited file: HasMaspoints: {file.Infos.HasMeasPoints}")
file.Close()

# Making a point grid

In [ ]:
def get_point_grid(diameter, grid_width=3, grid_height=3,addZ=False):
    # The center point is defined as 0,0
    center=[0,0]
    square_side = diameter
    points2D = []
    
    # Calculate the offset for centering the grid
    offset_x = center[0] -0.5 * square_side * (grid_width - 1)
    offset_y = center[1] -0.5 * square_side * (grid_height - 1)

    for i in range(grid_height):
        for j in range(grid_width):
            x = j * square_side + offset_x
            y = i * square_side + offset_y
            points2D.append([x, y])
    if addZ:
        points3D = [[x, y, 0] for x, y in points2D]
        return points3D
    return points2D

def plot_points(points):
    x, y = zip(*points)
    plt.scatter(x, y, color='red', marker='o')
    plt.grid()
    plt.show()
    
points = get_point_grid(diameter=3,grid_width=3,grid_height=3,addZ=False)
plot_points(points)